# Vehicle Detection Demo using LinearSVM

## Requirements:
- `opencv-python`
- `moviepy`

In [1]:
# Libraries
import matplotlib.image as mpimg
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.calibration import *
from skimage.feature import hog
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle
%matplotlib inline

## Parameters

In [2]:
# Parameters
color_space = 'YCrCb'
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL'
spatial_size = (24, 24)
hist_bins = 16
spatial_feat = True
hist_feat = True
hog_feat = True
window_parameters = [[(400,464),32,4],[(400,656),96,2],[(400,720),128,4]]

## Helper Functions

In [3]:
# Functions
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                        vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features

def grab_hog_from_patch(ypos, xpos, nblocks_per_window, hog_list):
    hog_feat1 = hog_list[0][ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
    hog_feat2 = hog_list[1][ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
    hog_feat3 = hog_list[2][ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
    return np.hstack((hog_feat1, hog_feat2, hog_feat3))

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1,color2,color3))

def color_hist(img, nbins=32):
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return hist_features

def color_mode(img, color_space):
    if color_space != 'RGB' and color_space != 'None':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)

    return feature_image

def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    features = []
    for file in imgs:
        image = mpimg.imread(file)
        file_features = single_img_features(image, color_space=color_space,
                        spatial_size=spatial_size, hist_bins=hist_bins,
                        orient=orient, pix_per_cell=pix_per_cell,
                        cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feat, hist_feat=hist_feat,
                        hog_feat=hog_feat, vis=False)
        features.append(file_features)
    
    return features

def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True,
                        vis=False):
    img_features = []
    feature_image = color_mode(img, color_space)
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        img_features.append(spatial_features)
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        img_features.append(hist_features)
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel],
                                    orient, pix_per_cell, cell_per_block,
                                    vis=False, feature_vec=True))
        else:
            if vis == True:
                hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel],
                               orient, pix_per_cell, cell_per_block, vis=True,
                               feature_vec=True)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel],
                               orient, pix_per_cell, cell_per_block, vis=False,
                               feature_vec=True)
        img_features.append(hog_features)

    if vis == True:
        return np.concatenate(img_features), hog_image
    else:
        return np.concatenate(img_features)

def apply_threshold(heatmap, threshold):
    heatmap = heatmap.copy()
    heatmap[heatmap <= threshold] = 0
    return heatmap

def draw_boxes(img, bboxes, color=(1, 0, 0), thick=6):
    imcopy = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], color=color, thickness=thick)
    return imcopy

def draw_labeled_bboxes(img, labels, color=(0, 255, 0)):
    for car_number in range(1, labels[1]+1):
        nonzero = (labels[0] == car_number).nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        cv2.rectangle(img, bbox[0], bbox[1], color, 6)
    return img

def slide_window(img_shape, x_start_stop=[None, None], y_start_stop=[None, None],
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img_shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img_shape[0]
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1
    window_list = []
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            window_list.append(((startx, starty), (endx, endy)))
    return window_list

def search_windows(img, windows, svm, scaler=None, color_space='RGB',
                    spatial_size=(32, 32), hist_bins=32,
                    hist_range=(0, 256), orient=9,
                    pix_per_cell=8, cell_per_block=2,
                    hog_channel=0, spatial_feat=True,
                    hist_feat=True, hog_feat=True):
    on_windows = []
    test_features = []
    for window in windows:
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        features = single_img_features(test_img, color_space=color_space,
                            spatial_size=spatial_size, hist_bins=hist_bins,
                            orient=orient, pix_per_cell=pix_per_cell,
                            cell_per_block=cell_per_block,
                            hog_channel=hog_channel, spatial_feat=spatial_feat,
                            hist_feat=hist_feat, hog_feat=hog_feat)
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        carpred = svm.predict(test_features)
        if carpred == 1:
            on_windows.append(window)

    return on_windows

def data_load():
    basedir = 'vehicles/'
    image_types = os.listdir(basedir)
    cars = []
    for imtype in image_types:
        cars.extend(glob.glob(basedir+imtype+'/*'))
    print('Number of Vehicle Images found:', len(cars))
    with open("cars.txt", 'w') as f:
        for fn in cars:
            f.write(fn+'\n')
    
    basedir = 'non-vehicles/'
    image_types = os.listdir(basedir)
    notcars = []
    for imtype in image_types:
        notcars.extend(glob.glob(basedir+imtype+'/*'))
    print('Number of Non-Vehicle Images found:', len(notcars))
    with open("notcars.txt", 'w') as f:
        for fn in notcars:
            f.write(fn+'\n')

    return cars, notcars

def find_cars(img, windowList, drawCount=False, svc=None):
    draw_img = np.copy(img)
    count = 0
    heatmap = np.zeros_like(img[:,:,0])
    im_tosearch = color_mode(img, color_space=color_space)
    im_tosearch = im_tosearch.astype(np.float32)/255
    for bound, window_size, cells_per_step in windowList:
        ctrans_tosearch = np.copy(im_tosearch)
        ystart = bound[0]
        ystop = bound[1]
        scale = window_size / 64
        window = 64
        ctrans_tosearch = ctrans_tosearch[ystart:ystop,:,:] # image sliced to ABS window zone
        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        hog_featuresbyChan = []
        for channel in range(ctrans_tosearch.shape[2]):
            hog_featuresbyChan.append(get_hog_features(ctrans_tosearch[:,:,channel], orient, pix_per_cell, cell_per_block, feature_vec=False))
        nxblocks = (ctrans_tosearch.shape[1] // pix_per_cell) - 1
        nyblocks = (ctrans_tosearch.shape[0] // pix_per_cell) - 1
        nblocks_per_window = (window // pix_per_cell) - 1
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step

        for xb in range(nxsteps):
            for yb in range(nysteps):
                count += 1
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                hog_features = grab_hog_from_patch(ypos, xpos, nblocks_per_window,
                                hog_featuresbyChan)
                xleft = xpos*pix_per_cell
                ytop = ypos*pix_per_cell
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window,
                        xleft:xleft+window], (64,64))
                spatial_features = bin_spatial(subimg, size=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)
                test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1,-1))
                test_predictions = svc.predict(test_features)
                if test_predictions == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    if drawCount == True:
                        cv2.rectangle(draw_img, (xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255))
                    heatmap[ytop_draw+ystart:ytop_draw+win_draw+ystart, xbox_left:xbox_left+win_draw] += 1

    if drawCount == True:
        return draw_img, heatmap, count
    else:
        return heatmap


## Model

In [4]:
def model():
    cars, notcars = data_load()
    t = time.time() # start timer
    car_features = extract_features(cars, color_space=color_space,
                                   spatial_size=spatial_size, hist_bins=hist_bins,
                                   orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block, hog_channel=hog_channel,
                                   spatial_feat=spatial_feat, hist_feat=hist_feat,
                                   hog_feat=hog_feat)

    notcar_features = extract_features(notcars, color_space=color_space,
                                   spatial_size=spatial_size, hist_bins=hist_bins,
                                   orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block, hog_channel=hog_channel,
                                   spatial_feat=spatial_feat, hist_feat=hist_feat,
                                   hog_feat=hog_feat)

    print(time.time()-t, 'Seconds to compute features...')

    X = np.vstack((car_features, notcar_features)).astype(np.float64)
    X_scaler = StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
    rand_state = np.random.randint(0,100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y,
                                    test_size=0.1, random_state=rand_state)
    print('Using:',orient,'orientations,',pix_per_cell,'pixels per cell',
         cell_per_block,'cells per block',hist_bins,'hostogram bins, and',
         spatial_size,'spatial sampling')
    print('Feature vector length:', len(X_train[0]))
    svc = LinearSVC()
    t = time.time()
    svc.fit(X_train, y_train)
    print(round(time.time()-t, 2), 'Seconds to train SVC...')
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

    return svc, X_scaler

## Pipeline

In [5]:
class Vehicle_Pipeline():
    def __init__(self, maximum, threshold, clf):
        self.detected = []
        self.maximum = maximum
        self.threshold = threshold
        self.clf = clf
        
    def process_image(self, image):
        heatmap_image = find_cars(image, window_parameters, svc=clf)
        tracking = None
        self.detected.append(heatmap_image)
        tracking = np.sum(self.detected, axis = 0)
        if len(self.detected) == self.maximum:
            tracking = apply_threshold(tracking, threshold=self.threshold)
            self.detected = self.detected[1:]
        labels = label(tracking)
        draw_img = draw_labeled_bboxes(np.copy(image), labels)
        return draw_img

# Get trained classifier
clf, X_scaler = model()

# Create vehicle detection processing object
detect_vehicles = Vehicle_Pipeline(maximum=18, threshold=8, clf=clf)

# Specify Video Clip Parameters
test_output = "project_video_output.mp4"
clip = VideoFileClip("project_video.mp4")
test_clip = clip.fl_image(detect_vehicles.process_image)

# Execute Video Processing
test_clip.write_videofile(test_output, audio=False)

Number of Vehicle Images found: 8792
Number of Non-Vehicle Images found: 8968
99.90547204017639 Seconds to compute features...
Using: 9 orientations, 8 pixels per cell 2 cells per block 16 hostogram bins, and (24, 24) spatial sampling
Feature vector length: 7068
29.28 Seconds to train SVC...
Test Accuracy of SVC =  0.9882
[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [19:35<00:00,  1.06it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

